# 任务3 特征工程&特征选择(3天)

# 特征工程

In [ ]:
#核心代码举例

# 统计特征
    #计算均值
    gp = train.groupby(by)[fea].mean()
    #计算中位数
    gp = train.groupby(by)[fea].median()
    #计算方差
    gp = train.groupby(by)[fea].std()
    #计算最大值
    gp = train.groupby(by)[fea].max()
    #计算最小值
    gp = train.groupby(by)[fea].min()
    #计算出现次数
    gp = train.groupby(by)[fea].size()
    

# groupby生成统计特征：mean,std
    # 按照communityName分组计算面积的均值和方差
    temp = data.groupby('communityName')['area'].agg({'com_area_mean': 'mean', 'com_area_std': 'std'})

# 特征拆分
    # 将houseType转为'Room'，'Hall'，'Bath'
    def Room(x):
        Room = int(x.split('室')[0])
        return Room
    def Hall(x):
        Hall = int(x.split("室")[1].split("厅")[0])
        return Hall
    def Bath(x):
        Bath = int(x.split("室")[1].split("厅")[1].split("卫")[0])
        return Bath

    data['Room'] = data['houseType'].apply(lambda x: Room(x))
    data['Hall'] = data['houseType'].apply(lambda x: Hall(x))
    data['Bath'] = data['houseType'].apply(lambda x: Bath(x))
    
#特征合并
    # 合并部分配套设施特征
    data['trainsportNum'] = 5 * data['subwayStationNum'] / data['subwayStationNum'].mean() + data['busStationNum'] / \
                                                                                             data[
                                                                                                 'busStationNum'].mean()

# 交叉生成特征:特征之间交叉+ - * / 
data['Room_Bath'] = (data['Bath']+1) / (data['Room']+1)


# 聚类特征
from sklearn.mixture import GaussianMixture  使用GaussianMixture做聚类特征
gmm = GaussianMixture(n_components=4, covariance_type='full', random_state=0)
gmm.fit_predict(data)
 
# 特征编码
from sklearn.preprocessing import LabelEncoder
data['communityName'] = LabelEncoder().fit_transform(data['communityName'])
from sklearn import preprocessing.OneHotEncoder
data['communityName'] = OneHotEncoder().fit_transform(data['communityName'])


# 过大量级值取log平滑（针对线性模型有效）
data[feature]=np.log1p(data[feature])

In [15]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

train = pd.read_csv('./data/FE_Housing_train.csv')
test = pd.read_csv('./data/test_a.csv')
target_train = train.pop('tradeMoney')
# target_test = test.pop('tradeMoney')


In [2]:
# 将houseType转为'Room'，'Hall'，'Bath'
def Room(x):
    Room = int(x.split('室')[0])
    return Room
def Hall(x):
    Hall = int(x.split("室")[1].split("厅")[0])
    return Hall
def Bath(x):
    Bath = int(x.split("室")[1].split("厅")[1].split("卫")[0])
    return Bath

train['Room'] = train['houseType'].apply(lambda x: Room(x))
train['Hall'] = train['houseType'].apply(lambda x: Hall(x))
train['Bath'] = train['houseType'].apply(lambda x: Bath(x))
train['Room_Bath'] = (train['Bath']+1) / (train['Room']+1)

In [4]:
train.head()

,ID,area,rentType,houseType,houseFloor,totalFloor,houseToward,houseDecoration,communityName,city,...,newWorkers,residentPopulation,pv,uv,lookNum,tradeTime,Room,Hall,Bath,Room_Bath
0,100309852,68.06,未知方式,2室1厅1卫,低,16,暂无数据,其他,XQ00051,SH,...,614,111546,1124.0,284.0,0,2018/11/28,2,1,1,0.666667
1,100307942,125.55,未知方式,3室2厅2卫,中,14,暂无数据,简装,XQ00130,SH,...,148,157552,701.0,22.0,1,2018/12/16,3,2,2,0.750000
2,100307764,132.00,未知方式,3室2厅2卫,低,32,暂无数据,其他,XQ00179,SH,...,520,131744,57.0,20.0,1,2018/12/22,3,2,2,0.750000
3,100306518,57.00,未知方式,1室1厅1卫,中,17,暂无数据,精装,XQ00313,SH,...,1665,253337,888.0,279.0,9,2018/12/21,1,1,1,1.000000
4,100305262,129.00,未知方式,3室2厅3卫,低,2,暂无数据,毛坯,XQ01257,SH,...,117,125309,2038.0,480.0,0,2018/11/18,3,2,3,1.000000


In [5]:
 # 填充租房类型
train.loc[(train['rentType'] == '未知方式') & (train['Room'] <=1), 'rentType'] = '整租'
train.loc[(train['rentType'] == '未知方式') & (train['Room_Bath'] <= 1), 'rentType'] = '合租' 
train.loc[(train['rentType'] == '未知方式') & (train['area'] / train['Room'] <= 20), 'rentType'] = '合租'

# 小于50平米的房子，如果超过1个房间，说明有隔断
train.loc[(train['rentType'] == '未知方式') & (train['Room'] > 1) & (train['area'] <=50), 'rentType'] = '合租'

train.loc[(train['rentType'] == '未知方式') & (train['area'] > 60) & (train['Room'] == 2), 'rentType'] = '整租'
train.loc[(train['rentType'] == '未知方式') & (train['area'] > 60) & (train['Room'] == 3), 'rentType'] = '整租'
train.loc[(train['rentType'] == '未知方式') & (train['area'] >= 100) & (train['Room'] > 3), 'rentType'] = '整租'
    
# 如果人均占比超过30平米
# train.loc[(train['rentType'] == '未知方式') & (train['area'] / train['Room'] >30 ), 'rentType'] = '整租'


In [6]:
a = train.loc[(train['area']) > 60]
aa = a.loc[a['rentType'] == '整租']

In [7]:
aa.sort_values(by='area')

,ID,area,rentType,houseType,houseFloor,totalFloor,houseToward,houseDecoration,communityName,city,...,newWorkers,residentPopulation,pv,uv,lookNum,tradeTime,Room,Hall,Bath,Room_Bath
36366,100008824,60.02,整租,1室1厅1卫,中,14,南,其他,XQ00936,SH,...,0,491767,39739.0,6001.0,0,2018/7/7,1,1,1,1.000000
10235,100223411,60.03,整租,1室1厅1卫,中,14,南,精装,XQ01751,SH,...,413,165159,4347.0,915.0,11,2018/11/2,1,1,1,1.000000
1434,100080491,60.05,整租,2室1厅1卫,中,5,西,精装,XQ01943,SH,...,0,187939,30251.0,2375.0,0,2018/2/22,2,1,1,0.666667
11852,100196687,60.05,整租,1室1厅1卫,中,6,南,其他,XQ01540,SH,...,0,248647,55639.0,6711.0,0,2018/5/11,1,1,1,1.000000
12255,100190481,60.05,整租,1室1厅1卫,低,6,南,其他,XQ01540,SH,...,0,248647,48990.0,5474.0,0,2018/7/25,1,1,1,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16877,100121838,689.67,整租,5室3厅6卫,低,3,南,精装,XQ03384,SH,...,0,97196,7381.0,1148.0,0,2018/9/19,5,3,6,1.166667
570,100072180,742.00,整租,4室3厅5卫,低,3,暂无数据,精装,XQ00508,SH,...,0,190706,51331.0,7105.0,0,2018/5/17,4,3,5,1.200000
34084,100020705,798.86,整租,5室3厅6卫,低,2,南,精装,XQ01187,SH,...,0,306857,22356.0,6545.0,0,2018/9/21,5,3,6,1.166667
1473,100052443,831.73,整租,4室3厅5卫,低,2,西,毛坯,XQ02102,SH,...,0,215146,12583.0,1257.0,0,2018/4/9,4,3,5,1.200000


In [10]:
def month(x):
    month = int(x.split('/')[1])
    return month

# 分割交易时间
train['month'] = train['tradeTime'].apply(lambda x: month(x))

# 合并配套设施特征, 给不同的配套设置不同的系数
train['trainsportNum'] = 5 * train['subwayStationNum'] / train['subwayStationNum'].mean() + train['busStationNum'] / train['busStationNum'].mean()
train['all_schoolNum'] = 2 * train['interSchoolNum'] / train['interSchoolNum'].mean() + train['schoolNum'] / train['schoolNum'].mean() \
                        + train['privateSchoolNum'] / train['privateSchoolNum'].mean()
train['all_hospitalNum'] = 2 * train['hospitalNum'] / train['hospitalNum'].mean() + train['drugStoreNum'] / train['drugStoreNum'].mean()
train['all_mall'] = train['mallNum'] / train['mallNum'].mean() + train['superMarketNum'] / train['superMarketNum'].mean()
train['otherNum'] = train['gymNum'] / train['gymNum'].mean() + train['bankNum'] / train['bankNum'].mean() + \
                   train['shopNum'] / train['shopNum'].mean() + 2 * train['parkNum'] / train['parkNum'].mean()

train.drop(['subwayStationNum', 'busStationNum',
           'interSchoolNum', 'schoolNum', 'privateSchoolNum',
           'hospitalNum', 'drugStoreNum', 'mallNum', 'superMarketNum', 'gymNum', 'bankNum', 'shopNum', 'parkNum'],
          axis=1, inplace=True)

train.drop('houseType', axis=1, inplace=True)
train.drop('tradeTime', axis=1, inplace=True)

train["area"] = train["area"].astype(int)

categorical_feats = ['rentType', 'houseFloor', 'houseToward', 'houseDecoration', 'communityName', 'region', 'plate']


## 特征合并

In [19]:
train['data_type'] = 0
test['data_type'] = 1
data = pd.concat([train, test], axis=0, join='outer')

In [32]:
def feature_count(data, features=[]):
    new_feature = 'count'
    for i in features:
        new_feature += '_' + i
    print(new_feature)
    temp = data.groupby(features).size().reset_index().rename(columns={0: new_feature})
    print(temp.head())
    data = data.merge(temp, 'left', on=features)
    return data

In [33]:
data = feature_count(data, ['communityName','totalFloor'])

count_communityName_totalFloor
  communityName  totalFloor  count_communityName_totalFloor
0       XQ00001          29                               1
1       XQ00002           7                               2
2       XQ00003           7                               1
3       XQ00004           7                               3
4       XQ00004          40                               1


In [37]:
data['count_communityName_totalFloor']

0         4
1         7
2        14
3        35
4         5
         ..
43904     1
43905     1
43906    46
43907     4
43908    14
Name: count_communityName_totalFloor, Length: 43909, dtype: int64

In [38]:
data['data_type']

0        0
1        0
2        0
3        0
4        0
        ..
43904    1
43905    1
43906    1
43907    1
43908    1
Name: data_type, Length: 43909, dtype: int64

## 计算统计特征

In [11]:
#计算统计特征
def featureCount(train,test):
    train['data_type'] = 0
    test['data_type'] = 1
    data = pd.concat([train, test], axis=0, join='outer')
    def feature_count(data, features=[]):
        new_feature = 'count'
        for i in features:
            new_feature += '_' + i
        temp = data.groupby(features).size().reset_index().rename(columns={0: new_feature})
        data = data.merge(temp, 'left', on=features)
        return data

    data = feature_count(data, ['communityName'])
    data = feature_count(data, ['buildYear'])
    data = feature_count(data, ['totalFloor'])
    data = feature_count(data, ['communityName', 'totalFloor'])
    data = feature_count(data, ['communityName', 'newWorkers'])
    data = feature_count(data, ['communityName', 'totalTradeMoney'])
    new_train = data[data['data_type'] == 0]
    new_test = data[data['data_type'] == 1]
    new_train.drop('data_type', axis=1, inplace=True)
    new_test.drop(['data_type'], axis=1, inplace=True)
    return new_train, new_test
    
train, test = featureCount(train, test)

## groupby方法生成统计特征

In [12]:
#groupby生成统计特征：mean,std等

def gourpby(train,test):
    train['data_type'] = 0
    test['data_type'] = 1
    data = pd.concat([train, test], axis=0, join='outer')
    columns = ['rentType', 'houseFloor', 'houseToward', 'houseDecoration', 'communityName', 'region', 'plate']
    for feature in columns:
        data[feature] = LabelEncoder().fit_transform(data[feature])

    temp = data.groupby('communityName')['area'].agg({'com_area_mean': 'mean', 'com_area_std': 'std'})
    temp.fillna(0, inplace=True)
    data = data.merge(temp, on='communityName', how='left')
    
    data['price_per_area'] = data.tradeMeanPrice / data.area * 100
    temp = data.groupby('communityName')['price_per_area'].agg(
        {'comm_price_mean': 'mean', 'comm_price_std': 'std'})
    temp.fillna(0, inplace=True)
    data = data.merge(temp, on='communityName', how='left')
   
    temp = data.groupby('plate')['price_per_area'].agg(
        {'plate_price_mean': 'mean', 'plate_price_std': 'std'})
    temp.fillna(0, inplace=True)
    data = data.merge(temp, on='plate', how='left')
    data.drop('price_per_area', axis=1, inplace=True)

    temp = data.groupby('plate')['area'].agg({'plate_area_mean': 'mean', 'plate_area_std': 'std'})
    temp.fillna(0, inplace=True)
    data = data.merge(temp, on='plate', how='left')
    
    temp = data.groupby(['plate'])['buildYear'].agg({'plate_year_mean': 'mean', 'plate_year_std': 'std'})
    data = data.merge(temp, on='plate', how='left')
    data.plate_year_mean = data.plate_year_mean.astype('int')
    data['comm_plate_year_diff'] = data.buildYear - data.plate_year_mean
    data.drop('plate_year_mean', axis=1, inplace=True)

    temp = data.groupby('plate')['trainsportNum'].agg('sum').reset_index(name='plate_trainsportNum')
    data = data.merge(temp, on='plate', how='left')
    temp = data.groupby(['communityName', 'plate'])['trainsportNum'].agg('sum').reset_index(name='com_trainsportNum')
    data = data.merge(temp, on=['communityName', 'plate'], how='left')
    data['trainsportNum_ratio'] = list(map(lambda x, y: round(x / y, 3) if y != 0 else -1,
                                           data['com_trainsportNum'], data['plate_trainsportNum']))
    data = data.drop(['com_trainsportNum', 'plate_trainsportNum'], axis=1)

    temp = data.groupby('plate')['all_SchoolNum'].agg('sum').reset_index(name='plate_all_SchoolNum')
    data = data.merge(temp, on='plate', how='left')
    temp = data.groupby(['communityName', 'plate'])['all_SchoolNum'].agg('sum').reset_index(name='com_all_SchoolNum')
    data = data.merge(temp, on=['communityName', 'plate'], how='left')
    data = data.drop(['com_all_SchoolNum', 'plate_all_SchoolNum'], axis=1)

    temp = data.groupby(['communityName', 'plate'])['all_mall'].agg('sum').reset_index(name='com_all_mall')
    data = data.merge(temp, on=['communityName', 'plate'], how='left')

    temp = data.groupby('plate')['otherNum'].agg('sum').reset_index(name='plate_otherNum')
    data = data.merge(temp, on='plate', how='left')
    temp = data.groupby(['communityName', 'plate'])['otherNum'].agg('sum').reset_index(name='com_otherNum')
    data = data.merge(temp, on=['communityName', 'plate'], how='left')
    data['other_ratio'] = list(map(lambda x, y: round(x / y, 3) if y != 0 else -1,
                                   data['com_otherNum'], data['plate_otherNum']))
    data = data.drop(['com_otherNum', 'plate_otherNum'], axis=1)

    temp = data.groupby(['month', 'communityName']).size().reset_index(name='communityName_saleNum')
    data = data.merge(temp, on=['month', 'communityName'], how='left')
    temp = data.groupby(['month', 'plate']).size().reset_index(name='plate_saleNum')
    data = data.merge(temp, on=['month', 'plate'], how='left')

    data['sale_ratio'] = round((data.communityName_saleNum + 1) / (data.plate_saleNum + 1), 3)
    data['sale_newworker_differ'] = 3 * data.plate_saleNum - data.newWorkers
    data.drop(['communityName_saleNum', 'plate_saleNum'], axis=1, inplace=True)

    new_train = data[data['data_type'] == 0]
    new_test = data[data['data_type'] == 1]
    new_train.drop('data_type', axis=1, inplace=True)
    new_test.drop(['data_type'], axis=1, inplace=True)
    return new_train, new_test

train, test = gourpby(train, test)

DataError: No numeric types to aggregate

## 聚类方法

In [13]:
#聚类
def cluster(train,test):
    from sklearn.mixture import GaussianMixture

    train['data_type'] = 0
    test['data_type'] = 1
    data = pd.concat([train, test], axis=0, join='outer')
    col = ['totalFloor',
           'houseDecoration', 'communityName', 'region', 'plate', 'buildYear',

           'tradeMeanPrice', 'tradeSecNum', 'totalNewTradeMoney',
           'totalNewTradeArea', 'tradeNewMeanPrice', 'tradeNewNum', 'remainNewNum',

           'landTotalPrice', 'landMeanPrice', 'totalWorkers',
           'newWorkers', 'residentPopulation', 'lookNum',
           'trainsportNum',
           'all_SchoolNum', 'all_hospitalNum', 'all_mall', 'otherNum']

    # EM
    gmm = GaussianMixture(n_components=3, covariance_type='full', random_state=0)
    data['cluster']= pd.DataFrame(gmm.fit_predict(data[col]))


    col1 = ['totalFloor','houseDecoration', 'communityName', 'region', 'plate', 'buildYear']
    col2 = ['tradeMeanPrice', 'tradeSecNum', 'totalNewTradeMoney',
            'totalNewTradeArea', 'tradeNewMeanPrice', 'tradeNewNum', 'remainNewNum',
            'landTotalPrice', 'landMeanPrice', 'totalWorkers',
            'newWorkers', 'residentPopulation', 'lookNum',
            'trainsportNum',
            'all_SchoolNum', 'all_hospitalNum', 'all_mall', 'otherNum']
    for feature1 in col1:
        for feature2 in col2:
        
            temp = data.groupby(['cluster',feature1])[feature2].agg('mean').reset_index(name=feature2+'_'+feature1+'_cluster_mean')
            temp.fillna(0, inplace=True)
       
            data = data.merge(temp, on=['cluster', feature1], how='left')
    
   
    new_train = data[data['data_type'] == 0]
    new_test = data[data['data_type'] == 1]
    new_train.drop('data_type', axis=1, inplace=True)
    new_test.drop(['data_type'], axis=1, inplace=True)
    
    return new_train, new_test

train, test = cluster(train, test)   

KeyError: "['all_SchoolNum'] not in index"

## log平滑

In [16]:
# 过大量级值取log平滑（针对线性模型有效）
big_num_cols = ['totalTradeMoney','totalTradeArea','tradeMeanPrice','totalNewTradeMoney', 'totalNewTradeArea',
                'tradeNewMeanPrice','remainNewNum', 'supplyNewNum', 'supplyLandArea',
                'tradeLandArea','landTotalPrice','landMeanPrice','totalWorkers','newWorkers',
                'residentPopulation','pv','uv']
for col in big_num_cols:
        train[col] = train[col].map(lambda x: np.log1p(x))
        test[col] = test[col].map(lambda x: np.log1p(x))
        

In [17]:
#对比特征工程前后线性模型结果情况
test=test.fillna(0)
# Lasso回归
from sklearn.linear_model import Lasso
lasso=Lasso(alpha=0.1)
lasso.fit(train,target_train)
#预测测试集和训练集结果
y_pred_train=lasso.predict(train)
y_pred_test=lasso.predict(test)

#对比结果
from sklearn.metrics import r2_score
score_train=r2_score(y_pred_train,target_train)
print("训练集结果：",score_train)
score_test=r2_score(y_pred_test, target_test)
print("测试集结果：",score_test)

ValueError: could not convert string to float: '未知方式'

# 特征选择

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
#读取数据
train = pd.read_csv('')
test = pd.read_csv('')

target_train = train.pop('tradeMoney')
target_test = test.pop('tradeMoney')

## 相关系数法

In [ ]:
#相关系数法特征选择
from sklearn.feature_selection import SelectKBest

print(train.shape)

sk=SelectKBest(k=150)
new_train=sk.fit_transform(train,target_train)
print(new_train.shape)

# 获取对应列索引
select_columns=sk.get_support(indices = True)
# print(select_columns)

# 获取对应列名
# print(test.columns[select_columns])
select_columns_name=test.columns[select_columns]
new_test=test[select_columns_name]
print(new_test.shape)
# Lasso回归
from sklearn.linear_model import Lasso

lasso=Lasso(alpha=0.1)
lasso.fit(new_train,target_train)
#预测测试集和训练集结果
y_pred_train=lasso.predict(new_train)

y_pred_test=lasso.predict(new_test)

#对比结果
from sklearn.metrics import r2_score
score_train=r2_score(y_pred_train,target_train)
print("训练集结果：",score_train)
score_test=r2_score(y_pred_test, target_test)
print("测试集结果：",score_test)

## Wrapper

In [ ]:
# Wrapper

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
rfe = RFE(lr, n_features_to_select=160)
rfe.fit(train,target_train)

RFE(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                               normalize=False),
    n_features_to_select=40, step=1, verbose=0)




select_columns = [f for f, s in zip(train.columns, rfe.support_) if s]
print(select_columns)
new_train = train[select_columns]
new_test = test[select_columns]

# Lasso回归
from sklearn.linear_model import Lasso

lasso=Lasso(alpha=0.1)
lasso.fit(new_train,target_train)
#预测测试集和训练集结果
y_pred_train=lasso.predict(new_train)

y_pred_test=lasso.predict(new_test)

#对比结果
from sklearn.metrics import r2_score
score_train=r2_score(y_pred_train,target_train)
print("训练集结果：",score_train)
score_test=r2_score(y_pred_test, target_test)
print("测试集结果：",score_test)

## Embedded
### 基于惩罚项的特征选择法
### Lasso(l1)和Ridge(l2)

In [ ]:
# Embedded
# 基于惩罚项的特征选择法
# Lasso(l1)和Ridge(l2)

from sklearn.linear_model import Ridge

ridge = Ridge(alpha=5)
ridge.fit(train,target_train)

Ridge(alpha=5, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

# 特征系数排序
coefSort = ridge.coef_.argsort()
print(coefSort)


# 特征系数
featureCoefSore=ridge.coef_[coefSort]
print(featureCoefSore)


select_columns = [f for f, s in zip(train.columns, featureCoefSore) if abs(s)> 0.0000005 ] 
# 选择绝对值大于0.0000005的特征

new_train = train[select_columns]
new_test = test[select_columns]
# Lasso回归
from sklearn.linear_model import Lasso

lasso=Lasso(alpha=0.1)
lasso.fit(new_train,target_train)
#预测测试集和训练集结果
y_pred_train=lasso.predict(new_train)

y_pred_test=lasso.predict(new_test)

#对比结果
from sklearn.metrics import r2_score
score_train=r2_score(y_pred_train,target_train)
print("训练集结果：",score_train)
score_test=r2_score(y_pred_test, target_test)
print("测试集结果：",score_test)

### 基于树模型的特征选择法
### 随机森林 平均不纯度减少（mean decrease impurity

In [ ]:
# Embedded
# 基于树模型的特征选择法
# 随机森林 平均不纯度减少（mean decrease impurity


from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
# 训练随机森林模型，并通过feature_importances_属性获取每个特征的重要性分数。rf = RandomForestRegressor()
rf.fit(train,target_train)
print("Features sorted by their score:")
print(sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), train.columns),
             reverse=True))

select_columns = [f for f, s in zip(train.columns, rf.feature_importances_) if abs(s)> 0.00005 ] 
# 选择绝对值大于0.00005的特征

new_train = train[select_columns]
new_test = test[select_columns]

# Lasso回归
from sklearn.linear_model import Lasso

lasso=Lasso(alpha=0.1)
lasso.fit(new_train,target_train)
#预测测试集和训练集结果
y_pred_train=lasso.predict(new_train)

y_pred_test=lasso.predict(new_test)

#对比结果
from sklearn.metrics import r2_score
score_train=r2_score(y_pred_train,target_train)
print("训练集结果：",score_train)
score_test=r2_score(y_pred_test, target_test)
print("测试集结果：",score_test)